<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_judge_answer_KCBERT_nsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질문에 대한 답변을 선별하는 코드(BERT Next Sentence Prediction)(로컬)

In [ ]:
!pip install soynlp
!pip install datasets
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from soynlp.word import WordExtractor
from tqdm import tqdm
from kobert_transformers import get_kobert_model, get_tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#모델 불러오기
HUGGINGFACE_MODEL_PATH = 'beomi/kcbert-base'

tokenizer = BertTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = BertForNextSentencePrediction.from_pretrained(HUGGINGFACE_MODEL_PATH)

In [ ]:
# #훈련 데이터 불러오기
# pretrain_dataset_path = '/content/drive/My Drive/KcBERT_pretrain_dataset.xlsx'
# finetune_dataset_path = '/content/drive/My Drive/KcBERT_finetune_dataset.xlsx'

# df_pretrain = df.read_excel(pretrain_dataset_path)
# df_finetune = df.read_excel(finetune_dataset_path)

In [ ]:
pretrain_dataset_path = '/content/drive/My Drive/KcBERT_pretrain_dataset.xlsx'
df_pretrain = df.read_excel(pretrain_dataset_path)

In [ ]:
#soynlp tokenizer를 Pretrain 데이터로 학습
pretrain_list = df_pretrain['text'].to_list()

word_extractor = WordExtractor()
word_extractor.train(pretrain_list)
word_score_table = word_extractor.extract()

In [ ]:
print(word_score_table)

In [ ]:
# 예제로 몇 가지 단어 점수 확인
for word, score in word_scores.items():
    print(f"{word}: {score.cohesion_forward:.2f}")

In [ ]:
# 높은 cohesion 점수를 가진 단어들을 선택하여 단어사전 구성
threshold = 0.1  # 점수 기준 설정
new_vocab = [word for word, score in word_scores.items() if score.cohesion_forward > threshold]

# 선택된 단어 출력
print(new_vocab)

In [ ]:
# KcBERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base')
model = BertModel.from_pretrained('beomi/kcbert-base')

# 새로운 단어 토크나이저에 추가
num_added_toks = tokenizer.add_tokens(new_vocab)
print(f"Added {num_added_toks} tokens")

# 모델의 임베딩 크기 조정
model.resize_token_embeddings(len(tokenizer))

In [ ]:
#카카오톡 데이터 불러오기
file_path = '/content/drive/My Drive/judge_question_result.csv'

In [ ]:
#카카오톡 대화내용을 데이터프레임으로 받기
df = pd.read_csv(file_path)

#질문 딕셔너리, 답변 목록 리스트(이중 리스트) 생성
questions, answer_lists = {}, []

#질문으로 판별된 텍스트를 새 데이터프레임으로 생성
df_question = df[df['label'] == 'question']

print(df_question)
#새 데이터프레임의 index, text를 question 딕셔너리에 저장
questions = {text : index for (index, text) in zip(df_question.index, df_question['text'])}
print(questions)

In [ ]:
# Ensure the classifier is using GPU
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise CPU
print(device)
classifier = pipeline('zero-shot-classification', model=HUGGINGFACE_MODEL_PATH, device=device, batch_size = 64)

In [ ]:
# 문장 정의
sentence_a = "안녕하세요. 저는 학생입니다."
sentence_b = "공부를 많이 하고 있어요."

# 인코딩 및 NSP 태스크 준비
encoded = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt')
with torch.no_grad():
    outputs = model(**encoded, next_sentence_label=torch.LongTensor([1]))
    logits = outputs.logits

# 결과 해석
predicted_label = torch.argmax(logits).item()

if predicted_label == 1:
    print("문장 B는 문장 A의 뒤에 올 수 있습니다.")
else:
    print("문장 B는 문장 A의 뒤에 올 수 없습니다.")


In [ ]:
#중복 질문 제거를 동시에 수행하기 위한 새로운 알고리즘

#실험을 위한 하이퍼파라미터 설정
#후보군으로 삼을 텍스트 개수 범위
text_range = 20
#답변 목록에 추가할지 기준이 되는 레이블값
#질문-질문 비교기준
qstandard = 0.6
#질문-답변 비교기준
astandard = 0.6

#질문-답변 쌍 딕셔너리 생성
qa_pair_dictionary = {index : {'질문' : question, '답변' : []} for index, question in zip(df_question.index, df_question['text'])}

#질문 속 질문인지 판별할 때 사용할 불리언
in_question_texts = False

all_texts = {index : item for index, item in zip(df.index, df['text'])}

for index, item in tqdm(all_texts.items(), desc = 'Processing Answer to Question'):

  candidate_qa_list = [] #현재 텍스트의 소속을 판정할 (질문-답변 딕셔너리) 리스트
  candidate_qa_index_list = []
  in_question_texts = True if df.iloc[index]['label'] == 'question' else False #판별할 텍스트가 질문인지 검사

  start = 0 if index < 20 else index - text_range - 1 #인덱스가 20 미만일 경우 검사 범위 조정
  for i in range(start, index): #현재 텍스트가 소속될 질문의 범위
    candidate = qa_pair_dictionary.get(i, None) #qa_pair_dictionary에서 i 인덱스에 해당하는 질답 딕셔너리 가져오기
    if candidate != None:
      candidate_qa_list.append(candidate) #결과 리스트에서 최대 확률인 질문을 인덱싱하기 위해 인덱스를 포함한 딕셔너리를 append
      candidate_qa_index_list.append(i)

  if len(candidate_qa_list) == 0:
    continue

  #데이터 배치처리
  sequence_to_classify = [qa_dict['질문'] + ' '.join(qa_dict['답변']) + '[END]' + item for qa_dict in candidate_qa_list] #[qa_dict['질문'] + '[END]' + item for qa_dict in candidate_qa_list] if in_question_texts == True else [qa_dict['질문'] + ' '.join(qa_dict['답변']) + '[END]' + item for qa_dict in candidate_qa_list]

  #동적 레이블
  candidate_labels = ["뒤의 질문은 앞의 글과 거의 같은 주제이다", "뒤의 질문은 앞의 글과 거의 같은 주제가 아니다"] if in_question_texts == True else ["질문-답변 쌍이다", "질문-답변 쌍이 아니다"] #"거의 같은 주제의 질문이다", "거의 같은 주제의 질문이 아니다"

  output = classifier(sequence_to_classify, candidate_labels, hypothesis_template = '[END]를 기준으로 두 질문을 구분했을 때 {}.' if in_question_texts == True else '[END]를 기준으로 두 글을 구분했을 때 {}.', multi_label=False)
  expected_label = "질문-답변 쌍이다" if in_question_texts == False else "뒤의 질문은 앞의 글과 거의 같은 주제이다"

  #output 딕셔너리에 질문 인덱스를 추가
  for i in range(len(output)):
    output[i]['index'] = candidate_qa_index_list[i]

  # 'scores'의 첫 번째 값(expected label일 확률)에 따라 내림차순으로 정렬
  sorted_output = sorted(output, key = lambda x : x['scores'][0], reverse=True)

  output = sorted_output[0] #텍스트가 소속될 질문

  #경우에 따른 기준값 변화 적용
  if (in_question_texts == True and output['labels'][0] == expected_label and output['scores'][0] > qstandard) or (in_question_texts == False and output['labels'][0] == expected_label and output['scores'][0] > astandard):
    #모순이 발생하지 않기 위해서 질문 속 질문으로 판별된 경우 즉시 qa_pair_dictionary에서 해당 질문을 삭제해야 함.
    if in_question_texts == True:
        del qa_pair_dictionary[index]

    qa_pair_dictionary[output['index']]['답변'].append(item)


  # print('\n',qa_pair_dictionary)
  # print('\n',used_question_index)


In [ ]:
print(answer_lists, end = '\n')

In [ ]:
# #데이터프레임 가공
# df_question.rename(columns = {'text' : 'question'})
# df_question['answer'] = answer_lists

data = {question : answer for question, answer in zip(questions.keys(), answer_lists)}
print(data)
data_result= [{'question' : key, 'answer' : value} for key, value in data.items()]
df_result = pd.DataFrame(data = data_result, columns = ['question', 'answer'])
print(df_result)

In [ ]:
#.csv 파일로 google drive에 저장
save_path = '/content/drive/My Drive/judge_answer_result.csv'

df_result.to_csv(save_path)